In [1]:
## Upload libraries
import sys
import pandas as pd
import numpy as np

#upload HY funcitons
from Functions.calc_func import * 
from Functions.upload_functions import * 

# export file path:
expath = 'C:/Users/Hatzav/OneDrive - University of Toronto/HY_UofT/002CarbonBudgeting/Code/CA_Const_EEIO/Data/csv_exports'

In [2]:
## IO Tables. (CIRAIG, CanadaIO V2.4, 2022) 2018 Statistics Canada data

# A	The technology matrix (16196 rows × 16196 columns) shows the normalized inter-industry trade. Read the information as "To produce 1$ of the product from the column, I need X$ of the product from the row"																	
															
# Y	The final demand matrix (16196 rows × 3627 columns) shows the demands from the different end-users. Read the information as "The consumer category from the column has bought X$ of the product in the row during the given year"																			
																				
# R	R (104 rows × 6396 columns) gives the factors of production (taxes, value-added, subsidies, etc.). Read the information as "To produce 1$ of the product from the column, I need X$ of factors of production from the row"																			
																				
# S	(11085 rows × 16196 columns) The normalized environmental extensions give the normalized direct emissions for each product. Read the information as "To produce 1$ of the product from the column, I emit Xkg of the pollutant from the row"																			
																				
# FY The direct emissions from the final demand gives the direct emissions from the different end-users, which are thus not directly linked to an economic activity (e.g., burning diesel in one's car). Read the information as "The consumer category from the column has directly emitted Xkg of pollutant from the row during the given year"																			
																				
# C	(33 rows × 11085 columns) The characterization matrix converts emissions of pollutants to impacts on the environment. Read the information as "The pollutant from the column has an impact of X on the impact category from the row"																			
																				
# L (16196 rows × 16196 columns) Liontief matrix

# FY (11085 rows × 3627 columns) - Direct emissions

In [2]:
#This cell loads all od CIRAIG's IO tables
# Tables use 2018 Stat Can. IO data, and openIO V2.4 EEIO model

#load Y table - final demand matrix (16196 rows × 3627)
#make sure to assign columns as indexes and headars, if not deafault

tables_Y = machine_loc('craig_Yfinal_demand.csv')
# load R matrix (104 rows × 6396 columns)
#tables_R = machine_loc('craig_R_norm_tech_matrix.csv',index_colf = [0,1], headerf = [0,1])
#load A matrix (16196 rows × 16196 columns)
#tables_A = machine_loc('craig_A_matrix.csv',index_colf = [0,1], headerf = [0,1])
#load L matrix (16196 rows × 16196 columns)
tables_L = machine_loc('craig_L_matrix.csv',index_colf = [0,1], headerf = [0,1])
#load S matrix (11085 rows × 16196 columns)
tables_S = machine_loc('craig_S_norm_env_flow_matrix.csv',index_colf = [0], headerf = [0,1])
#load C matrix (33 rows × 11085 columns)
tables_C = machine_loc('craig_C_characterization_matrix.csv',index_colf = [0,1], headerf = 0)
#load FY matrix (11085 rows × 3626 columns)
#tables_FY = machine_loc('craig_FY_direct_emissions_matrix.csv',index_colf = [0,1], headerf = [0,1,2])

# calculate (C)(S) table - environmental impact
tables_CS = tables_C.dot(tables_S)
print('Table CS calculated successfuly')

print('\n All tables loaded successfuly')

Couldnt load file on VM
loading craig_Yfinal_demand.csv from hard drive...
craig_Yfinal_demand.csv loaded successfuly
Couldnt load file on VM
loading craig_L_matrix.csv from hard drive...
craig_L_matrix.csv loaded successfuly
Couldnt load file on VM
loading craig_S_norm_env_flow_matrix.csv from hard drive...
craig_S_norm_env_flow_matrix.csv loaded successfuly
Couldnt load file on VM
loading craig_C_characterization_matrix.csv from hard drive...
craig_C_characterization_matrix.csv loaded successfuly
Table CS calculated successfuly

 All tables loaded successfuly


In [3]:
# Canada Province list s loaded.
provinces = ['CA-NL', 'CA-PE', 'CA-NS', 'CA-NB', 'CA-QC', 'CA-ON', 'CA-MB', 'CA-SK', 'CA-AB', 'CA-BC', 'CA-YT', 'CA-NT', 'CA-NU']
provinces

['CA-NL',
 'CA-PE',
 'CA-NS',
 'CA-NB',
 'CA-QC',
 'CA-ON',
 'CA-MB',
 'CA-SK',
 'CA-AB',
 'CA-BC',
 'CA-YT',
 'CA-NT',
 'CA-NU']

##  Calculate X total output of sector.

In [4]:
# this calculates the env impact for ()'CA-ON', 'Gross fixed capital formation, Construction')
#define province /sector /sub sector
province = 'CA-ON'
sector = 'Gross fixed capital formation, Construction'
sub_sector = 'Residential structures'
# this parameter will propegate as a headar to the env impact table
final_demand_category = (province, sector)
final_demand_category

#This line findes the final demand category to analyze (province, sector, sub sector)
on_GFCFC_Y  = tables_Y[(final_demand_category)]
# transformation to df requiered for dot() multiplication
on_GFCFC_Y  = pd.DataFrame(on_GFCFC_Y)
on_GFCFC_Y 

# 1. this line calc total output of sector. X = (L)16Kx16K * (Y)16Kx1
on_GFCFC_X = vector_X(tables_L, on_GFCFC_Y)
#call diagonolised X table. [0]  is the vector_X series, [1] is the diagonolized df

#2. Calculate environmental impact. returnes kgCO2e for called total output vector  
# call [1] for diagonal or [0] sector summary
on_GFCFC_CO2e = env_impact(tables_CS, on_GFCFC_X[1])

#3. assign title to column  
on_GFCFC_CO2e = pd.DataFrame(on_GFCFC_CO2e).rename(
    columns = {'Climate change, short term': final_demand_category})
on_GFCFC_CO2e

(CA-ON, Gross fixed capital formation, Construction)
                                                                                            kg CO2 eq (short)
CA-ON Ready-mixed concrete                                                                     1.910424e+09  
CN    Electricity by coal                                                                      9.533534e+08  
CA-ON Residential buildings                                                                    8.965944e+08  
      Concrete products                                                                        7.327438e+08  
CN    Basic iron and steel and of ferro-alloys and fi...                                       5.020737e+08  
...                                                                                                     ...  
US    Secondary glass for treatment, Re-processing of...                                       0.000000e+00  
CA-BC Other metal ores and concentrates                                                        0.000000e+00  
CA-ON Trusteed pension fund services                                                           0.000000e+00  
      Mutual funds (cost of service) and other simila...                                       0.000000e+00  
WM    Extra-territorial organizations and bodies                                               0.000000e+00  

[16196 rows x 1 columns]

In [6]:
#this block calculates the GCF construction total -output(X) for every province
#and appends it.
gfc_const_X = []

for province in provinces:
    # 1. defince Y vector
    gfcfc_Y = tables_Y[(province,'Gross fixed capital formation, Construction')].sum(1)
    #res_structY = pd.DataFrame(res_structY)
    # 2. Calculating encironemtnal impact using CIRIAGS's IO tables
    total_output = (tables_L).dot(gfcfc_Y)
    total_sum = total_output.sum()
    gfc_const_X.append(round(total_sum,0))
    print (str(total_sum) +' , is the total output (X) for GCF construction in ' + province)


12279628449.10872 , is the total output (X) for GCF construction in CA-NL
1616372998.458162 , is the total output (X) for GCF construction in CA-PE
10312930972.68797 , is the total output (X) for GCF construction in CA-NS
8636171976.411188 , is the total output (X) for GCF construction in CA-NB
109410276236.34576 , is the total output (X) for GCF construction in CA-QC
216082197878.02716 , is the total output (X) for GCF construction in CA-ON
22963418322.139572 , is the total output (X) for GCF construction in CA-MB
29978201792.889385 , is the total output (X) for GCF construction in CA-SK
132657818043.44041 , is the total output (X) for GCF construction in CA-AB
104176419289.08936 , is the total output (X) for GCF construction in CA-BC
1466182536.6492007 , is the total output (X) for GCF construction in CA-YT
1758236941.8665233 , is the total output (X) for GCF construction in CA-NT
2963135303.398187 , is the total output (X) for GCF construction in CA-NU


## Comparison DF GCFC, Res_building_, Res_X*0.27  

In [10]:
#1. this block calculates the GCF, Construction emissions for every province in MTCO2e
#and appends it to a list. CIRAIG's method
gcf_construction_MTCO2ciraig = list()
for province in provinces:
    # 1. defince Y vector
    gfc_const = tables_Y[(province,'Gross fixed capital formation, Construction')]
    
    # 2. Calculating encironemtnal impact using CIRIAGS's IO tables
    total_output = tables_C.dot(tables_S).dot(tables_L).dot(gfc_const)
    total_output

    ## 3. This lin sorts out the short term impacts of the selected data
    report_list1 = total_output.loc[('Climate change, short term','kg CO2 eq (short)')].T.sort_values(ascending=False).sum()
    report_list1 = round(report_list1.sum()/1000000000, 3)
        # Calculation Message
    print(province +' GFC Construction '+ str(report_list1) +' MTCO2e')
    gcf_construction_MTCO2ciraig.append(report_list1)
print (gcf_construction_MTCO2ciraig)

CA-NL GFC Construction 1.925 MTCO2e
CA-PE GFC Construction 0.223 MTCO2e
CA-NS GFC Construction 1.776 MTCO2e
CA-NB GFC Construction 1.156 MTCO2e
CA-QC GFC Construction 13.709 MTCO2e
CA-ON GFC Construction 26.824 MTCO2e
CA-MB GFC Construction 3.124 MTCO2e
CA-SK GFC Construction 5.487 MTCO2e
CA-AB GFC Construction 20.052 MTCO2e
CA-BC GFC Construction 14.49 MTCO2e
CA-YT GFC Construction 0.195 MTCO2e
CA-NT GFC Construction 0.292 MTCO2e
CA-NU GFC Construction 0.364 MTCO2e
[1.925, 0.223, 1.776, 1.156, 13.709, 26.824, 3.124, 5.487, 20.052, 14.49, 0.195, 0.292, 0.364]


In [11]:
#2. Calculating residential structures CO2e by provinve CIRAIG's method
res_structures_kgCO2_ciraig = list()
res_structures_MTCO2_ciraig = list()

for province in provinces:
    # 1. define Y vector
    gfc_res_const = tables_Y[(province,'Gross fixed capital formation, Construction','Residential structures')]
    
    # 2. Calculating encironemtnal impact using CIRIAGS's IO tables
    total_output = tables_C.dot(tables_S).dot(tables_L).dot(gfc_res_const)
    pd.DataFrame(total_output)

    ## 3. This lin sorts out the short term impacts of the selected data
    report_list1 = total_output.loc[('Climate change, short term','kg CO2 eq (short)')]
        # Calculation Message
    print (province +' GFC res structures '+ str(round(report_list1.sum()/1000000000, 3)) +' MTCO2e')
    res_structures_kgCO2_ciraig.append(report_list1)
    report_list2 = round(report_list1.sum()/1000000000, 3)
    res_structures_MTCO2_ciraig.append(report_list2)
    
#print (res_structures_kgCO2_ciraig)
#print (res_structures_MTCO2_ciraig)

CA-NL GFC res structures 0.284 MTCO2e
CA-PE GFC res structures 0.117 MTCO2e
CA-NS GFC res structures 0.976 MTCO2e
CA-NB GFC res structures 0.42 MTCO2e
CA-QC GFC res structures 6.843 MTCO2e
CA-ON GFC res structures 13.454 MTCO2e
CA-MB GFC res structures 1.174 MTCO2e
CA-SK GFC res structures 1.002 MTCO2e
CA-AB GFC res structures 5.755 MTCO2e
CA-BC GFC res structures 7.614 MTCO2e
CA-YT GFC res structures 0.046 MTCO2e
CA-NT GFC res structures 0.028 MTCO2e
CA-NU GFC res structures 0.01 MTCO2e
[283936385.29872054, 117284804.49322955, 975619224.6277148, 419608864.9393575, 6843331392.03512, 13454346663.20655, 1174125539.2008026, 1002301604.113766, 5755458573.635958, 7613598362.253084, 46127883.82277209, 27862485.522229634, 9576523.922741232]
[0.284, 0.117, 0.976, 0.42, 6.843, 13.454, 1.174, 1.002, 5.755, 7.614, 0.046, 0.028, 0.01]


In [12]:
# 3.this block calculates the GCF - residential structures total-output(Y) for every province
#and appends it.
residential_structures_X = []

for province in provinces:
    # 1. defince Y vector
    res_structY = tables_Y[(province,'Gross fixed capital formation, Construction','Residential structures')]
    total_sum = res_structY.sum()
    # 2. Calculating encironemtnal impact using CIRIAGS's IO tables
    #total_output = (tables_L).dot(res_structY)
    #total_sum = total_output.sum()
    residential_structures_X.append(round(total_sum,0))
    #print (str(total_sum) +' , ' + province)

    #Creatte a list, devide by 0.27, transform to MTCO2e and append to list
    # based on industry emissions in canada from: https://www150.statcan.gc.ca/t1/tbl1/en/cv.action?pid=3810009801
res_struct_MTCO2_statca = []
for i in residential_structures_X:
    x  = round(i*.27/1000000000,2)
    res_struct_MTCO2_statca.append(x)

#print (residential_structures_X) 
#print (res_struct_MTCO2_statca)

[1107094757.0, 444015434.0, 2774141254.0, 1675252013.0, 26828108716.0, 60419424940.0, 4306477504.0, 3051688297.0, 18256823569.0, 28269738534.0, 158235601.0, 88382990.0, 33885628.0]
[0.3, 0.12, 0.75, 0.45, 7.24, 16.31, 1.16, 0.82, 4.93, 7.63, 0.04, 0.02, 0.01]


In [13]:
#this block creates a df from of that compares CIRAIG's  emission
#model to Stat CA emission factor for residential construction by province
#lists
provinces
gcf_construction_MTCO2ciraig
res_structures_MTCO2_ciraig
res_struct_MTCO2_statca

#dictionary of lists

dict = {'total construction mtco2e CIRAIG':gcf_construction_MTCO2ciraig,
        'residential structures MTCO2e CIRAIG': res_structures_MTCO2_ciraig,
        'residential_structures_MTCO2e_StatCA': res_struct_MTCO2_statca
        }
#create dataframe from lists
prov_emissions = pd.DataFrame(dict, index = provinces)
## transform province emissions to MTCO2e 

#df['res_struct_MYCO2e_StatCA'] = df['residential structures CO2e']
prov_emissions


,total construction mtco2e CIRAIG,residential structures MTCO2e CIRAIG,residential_structures_MTCO2e_StatCA
CA-NL,1.925,0.284,0.30
CA-PE,0.223,0.117,0.12
CA-NS,1.776,0.976,0.75
CA-NB,1.156,0.420,0.45
CA-QC,13.709,6.843,7.24
CA-ON,26.824,13.454,16.31
CA-MB,3.124,1.174,1.16
CA-SK,5.487,1.002,0.82
CA-AB,20.052,5.755,4.93
CA-BC,14.490,7.614,7.63


In [15]:
prov_emissions.to_csv(expath + 'province_construction_emissions_ciraig.csv')

#### Calc residential_constrution CO2e/m2 for ON in 2018

In [18]:
res_structures_MTCO2_ciraig[5]

13.454

## Emission/m2 total residential construction (top-down approach)

In [22]:
# find emissions/residential_constrution m2 in ontario
# the emission are devided by the 26.1 M sqm of residential building built in Canada. source is
# https://oee.nrcan.gc.ca/corporate/statistics/neud/dpa/showTable.cfm?type=CP&sector=res&juris=on&rn=18&page=0
# difference between total floor space built between 2018-2019
on_co2_m2 = round(res_structures_MTCO2_ciraig[5].sum()/26100000*1000000000, 0)
on_co2_m2

515.0